In [1]:
import DataPreparation as dpr
import ModelRun as mr
# import benchmarks as bench

import os
import itertools
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.model_selection import TimeSeriesSplit

from MyEstimators import CLS_Estimator

### Load data

In [2]:
df = dpr.read_data('EQP_Quarterly')
df = dpr.data_clean(df, '1956-01-01')

In [3]:
df.head()

,EQP,DP,DY,EP,DE,svar,b/m,ntis,tbl,lty,...,TMS,DFR,DFY,infl,c,w,y,cay,AAA,BAA
time,,,,,,,,,,,,,,,,,,,,,
1956-03-01,0.066512,-3.333030,-3.269151,-2.575525,-0.757505,0.003289,0.531077,0.026695,0.0225,0.0303,...,0.0078,0.005159,0.0050,0.000000,9.272498,11.092725,9.100386,0.007275,0.0310,0.0360
1956-06-01,-0.028264,-3.261722,-3.293365,-2.568575,-0.693147,0.003688,0.551565,0.025672,0.0249,0.0299,...,0.0050,-0.021824,0.0050,0.014925,9.271728,11.091665,9.107828,0.000775,0.0326,0.0376
1956-09-01,-0.034415,-3.204645,-3.239744,-2.573142,-0.631503,0.002519,0.571910,0.029362,0.0284,0.0324,...,0.0040,0.005663,0.0051,0.007353,9.269304,11.086198,9.106428,0.000663,0.0356,0.0407
1956-12-01,0.033241,-3.289216,-3.260525,-2.616389,-0.672827,0.004394,0.544177,0.026149,0.0321,0.0345,...,0.0024,-0.002208,0.0062,0.007299,9.277993,11.096678,9.118405,-0.002524,0.0375,0.0437
1957-03-01,-0.050750,-3.238565,-3.294980,-2.562911,-0.675654,0.002288,0.599819,0.026600,0.0308,0.0331,...,0.0023,-0.000368,0.0077,0.007246,9.280482,11.090721,9.117433,0.002041,0.0366,0.0443


### Add $y_{t-1}$ and construct X and y

In [4]:
df['y_lag'] = df['EQP'].shift()
df = df.dropna()
df.head()

,EQP,DP,DY,EP,DE,svar,b/m,ntis,tbl,lty,...,DFR,DFY,infl,c,w,y,cay,AAA,BAA,y_lag
time,,,,,,,,,,,,,,,,,,,,,
1956-06-01,-0.028264,-3.261722,-3.293365,-2.568575,-0.693147,0.003688,0.551565,0.025672,0.0249,0.0299,...,-0.021824,0.0050,0.014925,9.271728,11.091665,9.107828,0.000775,0.0326,0.0376,0.066512
1956-09-01,-0.034415,-3.204645,-3.239744,-2.573142,-0.631503,0.002519,0.571910,0.029362,0.0284,0.0324,...,0.005663,0.0051,0.007353,9.269304,11.086198,9.106428,0.000663,0.0356,0.0407,-0.028264
1956-12-01,0.033241,-3.289216,-3.260525,-2.616389,-0.672827,0.004394,0.544177,0.026149,0.0321,0.0345,...,-0.002208,0.0062,0.007299,9.277993,11.096678,9.118405,-0.002524,0.0375,0.0437,-0.034415
1957-03-01,-0.050750,-3.238565,-3.294980,-2.562911,-0.675654,0.002288,0.599819,0.026600,0.0308,0.0331,...,-0.000368,0.0077,0.007246,9.280482,11.090721,9.117433,0.002041,0.0366,0.0443,0.033241
1957-06-01,0.075114,-3.309868,-3.238565,-2.628349,-0.681519,0.001363,0.565877,0.030528,0.0329,0.0361,...,-0.003789,0.0072,0.010791,9.278119,11.104916,9.118823,-0.004528,0.0391,0.0463,-0.050750


In [5]:
X = df[['DP','DY']]
station = pd.DataFrame()
y = df[['EQP']].squeeze()
X.head(2)

,DP,DY
time,,
1956-06-01,-3.261722,-3.293365
1956-09-01,-3.204645,-3.239744


### Construct single-index and nonlinear models

In [6]:
def single_index(x):
    if isinstance(x, (pd.DataFrame, np.ndarray)):
        if isinstance(x, pd.DataFrame):
            x_values = x.values
        else:
            pass
    else:
        raise Exception('wrong type')

    def u(theta):
        if len(theta) == x_values.shape[1]:
            sum_up = [x_values[:, i] * theta[i] for i in range(x_values.shape[1])]
            index = np.sum(sum_up, axis=0)
        else:
            raise Exception('wrong parameter dimension')
        return index

    return u

In [7]:
extra_params = {'sin_func':1,
               'cos_func':1,
               'scaled_sin_func':2,
               'scaled_cos_func':2,
               'exp_func':2,
               'exp_shift_func':2,
                'poly_func':3
               }

In [8]:
def dimensions(non_sta, sta, func):
    stas = sta.shape[1]
    nonstas = non_sta.shape[1]
    extra = range(0, extra_params[func])
    return nonstas, stas, extra

In [9]:
def sin_func(x):
    def objective_func(params):
        func = np.sin(single_index(x.iloc[:,:d1])(params[0:d1])+params[d1+d2+extra[0]])+np.dot(
            x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [10]:
def cos_func(x):
    def objective_func(params):
        func = np.cos(single_index(x.iloc[:,:d1])(params[0:d1])+params[d1+d2+extra[0]])+np.dot(
            x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [11]:
def scaled_sin_func(x):
    def objective_func(params):
        func = np.sin(params[d1+d2+extra[1]]*single_index(x.iloc[:,:d1])(
            params[0:d1])+params[d1+d2+extra[0]])+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [12]:
def scaled_cos_func(x):
    def objective_func(params):
        func = np.cos(params[d1+d2+extra[1]]*single_index(x.iloc[:,:d1])(
            params[0:d1])+params[d1+d2+extra[0]])+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [13]:
def exp_shift_func(x):
    def objective_func(params):
        func = 1 - np.exp(params[d1+d2+extra[1]]*((single_index(x.iloc[:,:d1])(
            params[0:d1]))-params[d1+d2+extra[0]])**2)+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [14]:
def exp_func(x):
    def objective_func(params):
        func = params[d1+d2+extra[0]]*np.exp(-params[d1+d2+extra[1]]*(single_index(x.iloc[:,:d1])(params[0:d1]))**2
                                )+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [15]:
def poly_func(x):
    def objective_func(params):
        func = params[d1+d2+extra[0]]+params[d1+d2+extra[1]]*(single_index(x.iloc[:,:d1])(
            params[0:d1]))+params[d1+d2+extra[2]]*((single_index(x.iloc[:,:d1])(
            params[0:d1]))**2)+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
#                (single_index(x.iloc[:,:d1])(params[0:d1])
        return func
    return objective_func

### Model Estimation

In [16]:
def constraint_func(x):
    def constraint(params):
        con = 0
        for j in np.arange(0, x.iloc[:,:d1].shape[1]):
            con += params[j]**2
            cons = con - 1
        return cons
    return {'type':'eq', 'fun': constraint}

In [17]:
d1, d2, extra = dimensions(X, station,'poly_func')
cls_nls = CLS_Estimator(obj_func = poly_func, x0 = [0.001]*(d1+d2+extra[-1]+1))
cls = CLS_Estimator(obj_func = poly_func, x0 = [0.001]*(d1+d2+extra[-1]+1), constraints = constraint_func(X))

In [18]:
cls_nls.fit(X,y)
cls.fit(X,y)

CLS_Estimator(constraints={'fun': <function constraint_func.<locals>.constraint at 0x000001CF668881E0>,
                           'type': 'eq'},
              obj_func=<function poly_func at 0x000001CF668869D8>,
              x0=[0.001, 0.001, 0.001, 0.001, 0.001])

In [19]:
print(cls_nls.params_)
print(cls.params_)

[-8.94841330e-04  1.92702711e-03  1.24399499e-02 -2.56973660e-05
  1.01218548e-03]
[-0.16930924  0.98556298  0.00341686 -0.03364998 -0.01037493]


In [20]:
cls.params_[0]

-0.16930923753093413

In [21]:
cls.params_[0]**2+cls.params_[1]**2

1.0000000010637105

## Empirical Study

### Cointegrated predictors
- dividend-price ratio and dividend yield
- T-bill rate and long-term yield
- dividend-price ratio and earningprice ratio
- baa- and aaa-rated corporate bond yields

In [22]:
co1 = df[['DP', 'DY']]
co2 = df[['tbl', 'lty']]
co3 = df[['DP', 'EP']]
co4 = df[['BAA', 'AAA']]

In [23]:
cointe_variables = [co1, co2, co3, co4]

### Stationary variables

In [24]:
station = df[['y_lag', 'cay']]

### Fit model and Save Results

In [25]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [26]:
fun_list = [sin_func,
            cos_func,
            scaled_sin_func,
            scaled_cos_func,
            exp_func,
            exp_shift_func,
            poly_func
           ]

In [27]:
# Set up hierachical index
fun_names = [i.__name__ for i in fun_list]
cointe_names = [get_df_name(i) for i in cointe_variables]
iterables_a = [fun_names, cointe_names]

In [28]:
#Set up directory
parent = os.getcwd()
folder = 'results'
path = os.path.join(parent, folder)
if not os.path.exists(path):
    os.makedirs(path)

In [29]:
results = pd.DataFrame()
for i, j in itertools.product(fun_list, cointe_variables):
    # Set up dimensions
    d1, d2, extra= dimensions(j,station, i.__name__)
    initial_len = d1+d2+extra[-1]+1
    
    # Set up dataframes
    iterables = [[i.__name__], [get_df_name(j)]]
    sec_columns = ['param_'+str(i) for i in range(1,initial_len+1)]
    multi_index = pd.MultiIndex.from_product(iterables, names=["function", "variables"])
    multi_columns = pd.MultiIndex.from_product([['NLS', 'CLS'], sec_columns],
                                               names=['Estimator', 'Parameters'])
    result = pd.DataFrame(index = multi_index, columns = multi_columns)
    # Prepare X
    X_ = j.join(station)
    # Fit models
    nls = CLS_Estimator(obj_func = i, x0 = [0.001]*initial_len)
    cls = CLS_Estimator(obj_func = i, x0 = [0.001]*initial_len, constraints = constraint_func(X_))
    nls.params_ = nls.fit(X_,y).params_
    cls.params_ = cls.fit(X_,y).params_
    # Save results to dataframe
    result.loc[i.__name__,get_df_name(j)].loc['NLS'] = nls.params_ 
    result.loc[i.__name__,get_df_name(j)].loc['CLS'] = cls.params_ 
    # Put into one table
    results = results.append(result, ignore_index = False, sort = False)
    
# Export to Excel
results.to_excel('Results/full_sample.xlsx')

In [30]:
results.head(3)

Estimator                CLS                                                   \
Parameters           param_1   param_2   param_3   param_4    param_5 param_6   
function variables                                                              
sin_func co1        0.709838 -0.704365  0.022517 -0.104848 -15.689339     NaN   
         co2        0.495116 -0.868827  0.079209  0.054524 -50.222284     NaN   
         co3        0.715437 -0.698678 -0.221696 -0.790349  -2.565328     NaN   

Estimator                        NLS                                          \
Parameters         param_7   param_1   param_2   param_3   param_4   param_5   
function variables                                                             
sin_func co1           NaN -1.008436  1.002847  0.000839 -0.073964 -0.023462   
         co2           NaN -0.786401  0.690315  0.066833 -0.247875  0.003686   
         co3           NaN -0.026356  -0.00009  0.059006 -0.224006 -0.083221   

Estimator                           
Parameters         param_6 param_7  
function variables                  
sin_func co1           NaN     NaN  
         co2           NaN     NaN  
         co3           NaN     NaN

# WTF is hapenning here?!!!

In [31]:
# def fit_model(functions, variables, file_name):   
#     results = pd.DataFrame()
#     for i, j in itertools.product(functions, variables):
#         # Set up dimensions
#         d1, d2, extra= dimensions(j,station, i.__name__)
# #         print(d1,d2,extra[-1])
#         initial_len = d1+d2+extra[-1]+1

#         # Set up dataframes
#         iterables = [[i.__name__], [get_df_name(j)]]
#         sec_columns = ['param_'+str(i) for i in range(1,initial_len+1)]
#         multi_index = pd.MultiIndex.from_product(iterables, names=["function", "variables"])
#         multi_columns = pd.MultiIndex.from_product([['NLS', 'CLS'], sec_columns],
#                                                    names=['Estimator', 'Parameters'])
#         result = pd.DataFrame(index = multi_index, columns = multi_columns)
#         # Prepare X
#         X_ = j.join(station)
#         # Fit models
#         nls = CLS_Estimator(obj_func = i, x0 = [0.001]*initial_len)
#         cls = CLS_Estimator(obj_func = i, x0 = [0.001]*initial_len, constraints = constraint_func(X))
#         nls.params_ = nls.fit(X_,y).params_
#         cls.params_ = cls.fit(X_,y).params_
#         print(initial_len)
#         # Save results to dataframe
#         result.loc[i.__name__,get_df_name(j)].loc['NLS'] = nls.params_ 
#         result.loc[i.__name__,get_df_name(j)].loc['CLS'] = cls.params_ 
#         # Put into one table
#         results = results.append(result, ignore_index = False, sort = False)
#     # Export to Excel
#     path = 'Results/' + file_name +'.xlsx'
#     results.to_excel(path)
#     return results

In [32]:
# fit_model(fun_list, cointe_variables,'results_1835')

## Use initial values from Linear regression (using Taylor expansion)

In [33]:
from sklearn.linear_model import LinearRegression

### Initial values for $\theta$

In [34]:
LR = LinearRegression()

In [35]:
LR_theta = LR.fit(co1.iloc[:,1:], co1.iloc[:,:1])

In [36]:
alpha = np.append(1,LR_theta.coef_)
theta = np.array(-alpha/np.linalg.norm(alpha))
theta

array([-0.7138883 , -0.70025959])

### Initial values for $\beta$ and $\gamma$

#### 1st order Taylor

In [37]:
station = df[['y_lag', 'cay']]

In [38]:
u = single_index(co1)(theta)

In [39]:
station.head(3)

,y_lag,cay
time,,
1956-06-01,0.066512,0.000775
1956-09-01,-0.028264,0.000663
1956-12-01,-0.034415,-0.002524


In [40]:
X_order1 = station.copy()
X_order1['u'] = u
X_order1.head(3)

,y_lag,cay,u
time,,,
1956-06-01,0.066512,0.000775,4.634716
1956-09-01,-0.028264,0.000663,4.556420
1956-12-01,-0.034415,-0.002524,4.631347


In [41]:
LR_O1 = LR.fit(X_order1, y)

#### Initials for sin_func and scaled_sin_func

In [42]:
ini_sin = np.append(np.append(theta,LR_O1.coef_[:d2]), LR_O1.intercept_)
d = {'sin_func':ini_sin}
ini_sin = pd.DataFrame(d)
ini_sin

,sin_func
0,-0.713888
1,-0.700260
2,0.074586
3,-0.175148
4,-0.008753


In [43]:
ini_scaled_sin = np.append(np.append(theta, LR_O1.coef_[:d2]),([LR_O1.intercept_], [LR_O1.coef_[0]]))
ini_scaled_sin

array([-0.7138883 , -0.70025959,  0.07458612, -0.17514771, -0.00875318,
        0.07458612])

In [44]:
d = {'scaled_sin_func':ini_scaled_sin}
ini_scaled_sin = pd.DataFrame(d)
ini_scaled_sin

,scaled_sin_func
0,-0.713888
1,-0.700260
2,0.074586
3,-0.175148
4,-0.008753
5,0.074586


#### 2nd order Taylor

In [45]:
X_order2 = X_order1
X_order2['u2'] = u**2
X_order2.head(3)

,y_lag,cay,u,u2
time,,,,
1956-06-01,0.066512,0.000775,4.634716,21.480589
1956-09-01,-0.028264,0.000663,4.556420,20.760961
1956-12-01,-0.034415,-0.002524,4.631347,21.449376


In [46]:
LR_O2 = LR.fit(X_order2, y)

#### Initials for cos_func and scaled_cos_func

In [47]:
ini_cos = np.append(np.append(theta,LR_O2.coef_[:d2]), [-LR_O2.coef_[d2]])
d = {'cos_func':ini_cos}
ini_cos = pd.DataFrame(d)
ini_cos

,cos_func
0,-0.713888
1,-0.700260
2,0.068108
3,-0.148068
4,-0.119544


In [48]:
ini_scaled_cos = np.append(np.append(theta, LR_O2.coef_[:d2]),([-LR_O2.coef_[2]/np.sqrt(
    -2*LR_O2.coef_[3])], [np.sqrt(-2*LR_O2.coef_[3])]))
ini_scaled_cos

array([-0.7138883 , -0.70025959,  0.0681076 , -0.14806838, -0.79623137,
        0.15013681])

In [49]:
d = {'scaled_cos_func':ini_scaled_cos}
ini_scaled_cos = pd.DataFrame(d)
ini_scaled_cos

,scaled_cos_func
0,-0.713888
1,-0.700260
2,0.068108
3,-0.148068
4,-0.796231
5,0.150137


#### Initials for poly_func

In [50]:
ini_poly = np.append(np.append(theta, LR_O2.coef_[:d2]),LR_O2.coef_[d2:])
ini_poly = np.insert(ini_poly, 4, LR_O2.intercept_)
ini_poly

array([-0.7138883 , -0.70025959,  0.0681076 , -0.14806838, -0.3015171 ,
        0.11954364, -0.01127053])

In [51]:
d = {'poly_func':ini_poly}
ini_poly = pd.DataFrame(d)
ini_poly

,poly_func
0,-0.713888
1,-0.700260
2,0.068108
3,-0.148068
4,-0.301517
5,0.119544
6,-0.011271


#### 4st Taylor

In [52]:
X_order4 = X_order2
X_order4['u3'], X_order4['u4'] = u**3, u**4
X_order4.head(3)

,y_lag,cay,u,u2,u3,u4
time,,,,,,
1956-06-01,0.066512,0.000775,4.634716,21.480589,99.556425,461.415725
1956-09-01,-0.028264,0.000663,4.556420,20.760961,94.595654,431.017506
1956-12-01,-0.034415,-0.002524,4.631347,21.449376,99.339504,460.075723


In [53]:
LR_O4 = LR.fit(X_order4, y)

In [54]:
gamma = [-LR_O4.coef_[2]/2*LR_O4.coef_[3], LR_O4.coef_[3]]
gamma

[1.489849622076274, -1.0219284963691389]

#### Initials for exp_shift_func

In [55]:
ini_exp_shift = np.append(np.append(theta, LR_O4.coef_[:d2]), gamma)
d = {'exp_shift_func':ini_exp_shift}
ini_exp_shift = pd.DataFrame(d)
ini_exp_shift

,exp_shift_func
0,-0.713888
1,-0.700260
2,0.068523
3,-0.110023
4,1.489850
5,-1.021928


#### 6st Taylor

In [56]:
X_order6 = X_order4
X_order6['u6'] = u**6
X_order6 = X_order6.drop(['u','u3'], axis = 1)

In [57]:
X_order6.head(3)

,y_lag,cay,u2,u4,u6
time,,,,,
1956-06-01,0.066512,0.000775,21.480589,461.415725,9911.481777
1956-09-01,-0.028264,0.000663,20.760961,431.017506,8948.337673
1956-12-01,-0.034415,-0.002524,21.449376,460.075723,9868.337099


In [58]:
LR_O6 = LR.fit(X_order6, y)

In [59]:
gamma = [LR_O6.intercept_, -LR_O4.coef_[2]/LR_O6.intercept_]
gamma

[0.28827362829118247, 0.12687223534386796]

#### Initials for exp_func

In [60]:
ini_exp = np.append(np.append(theta, LR_O6.coef_[:d2]), gamma)
d = {'exp_func':ini_exp}
ini_exp = pd.DataFrame(d)
ini_exp

,exp_func
0,-0.713888
1,-0.700260
2,0.068683
3,-0.108124
4,0.288274
5,0.126872


In [61]:
dfs = [ini_sin, ini_cos, ini_scaled_sin, ini_scaled_cos, ini_exp_shift, ini_exp, ini_poly]
taylor_init = pd.concat(dfs, axis=1)
taylor_init

,sin_func,cos_func,scaled_sin_func,scaled_cos_func,exp_shift_func,exp_func,poly_func
0,-0.713888,-0.713888,-0.713888,-0.713888,-0.713888,-0.713888,-0.713888
1,-0.700260,-0.700260,-0.700260,-0.700260,-0.700260,-0.700260,-0.700260
2,0.074586,0.068108,0.074586,0.068108,0.068523,0.068683,0.068108
3,-0.175148,-0.148068,-0.175148,-0.148068,-0.110023,-0.108124,-0.148068
4,-0.008753,-0.119544,-0.008753,-0.796231,1.489850,0.288274,-0.301517
5,NaN,NaN,0.074586,0.150137,-1.021928,0.126872,0.119544
6,NaN,NaN,NaN,NaN,NaN,NaN,-0.011271


### Fit model and Save Results

In [62]:
results_Taylor = pd.DataFrame()
for i, j in itertools.product(fun_list, cointe_variables):
    # Set up dimensions
    d1, d2, extra= dimensions(j,station, i.__name__)
    initial_len = d1+d2+extra[-1]+1
    # Set up dataframes
    iterables = [[i.__name__], [get_df_name(j)]]
    sec_columns = ['param_'+str(i) for i in range(1,initial_len+1)]
    multi_index = pd.MultiIndex.from_product(iterables, names=["function", "variables"])
    multi_columns = pd.MultiIndex.from_product([['NLS', 'CLS'], sec_columns],
                                               names=['Estimator', 'Parameters'])
    result = pd.DataFrame(index = multi_index, columns = multi_columns)
    # Prepare X
    X_ = j.join(station)
    # Fit models
    nls = CLS_Estimator(obj_func = i, x0 = taylor_init[i.__name__].dropna())
    cls = CLS_Estimator(obj_func = i, x0 = taylor_init[i.__name__].dropna(), constraints = constraint_func(X_))
    nls.params_ = nls.fit(X_,y).params_
    cls.params_ = cls.fit(X_,y).params_
    # Save results to dataframe
    result.loc[i.__name__,get_df_name(j)].loc['NLS'] = nls.params_ 
    result.loc[i.__name__,get_df_name(j)].loc['CLS'] = cls.params_ 
    # Put into one table
    results_Taylor = results_Taylor.append(result, ignore_index = False, sort = False)
    
# Export to Excel
results_Taylor.to_excel('Results/Taylor_fullsample.xlsx')
results_Taylor.tail()

Estimator                      CLS                                          \
Parameters                 param_1   param_2   param_3   param_4   param_5   
function       variables                                                     
exp_shift_func co4        0.910574 -0.413347  0.071831 -0.108883  0.103539   
poly_func      co1       -0.713889 -0.700259  0.068108 -0.148068 -0.301517   
               co2       -0.753511  0.657435  0.066707 -0.247565  0.004134   
               co3       -0.980008  0.198959  0.034296 -0.159542 -0.827409   
               co4        0.639797 -0.768544  0.028411  0.231277  0.030052   

Estimator                                             NLS                      \
Parameters                 param_6      param_7   param_1   param_2   param_3   
function       variables                                                        
exp_shift_func co4       -2.979366          NaN  2.551573  2.390207  0.074001   
poly_func      co1        0.119544    -0.011271 -0.713889 -0.700259  0.068108   
               co2        1.061836    -1.817768 -2.223548  1.981999  0.067084   
               co3        0.533388    -0.083326 -1.577195  0.321112  0.034126   
               co4         0.07019 -1093.419622  6.212309 -7.462416  0.028392   

Estimator                                                          
Parameters                 param_4   param_5   param_6    param_7  
function       variables                                           
exp_shift_func co4        -0.10841  1.014999 -0.026481        NaN  
poly_func      co1       -0.148068 -0.301517  0.119544  -0.011271  
               co2       -0.256923  0.004605   0.41972  -1.931044  
               co3        -0.15964 -0.829506  0.332488  -0.032298  
               co4        0.229646  0.030016  0.006881 -11.591215

### GridSearch and CrossValidation

### Train_test split

In [63]:
X_train, X_test, y_train, y_test = dpr.data_split(df, 'EQP', "1987-12-01", "2019-12-01")

In [64]:
val_length = 1
test_length = 1
step = 1
# cv_outer = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=int((12/step) * test_length), test_size=step)
# cv_inner = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=int((12/step) * val_length), test_size=step)
cv_outer = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=step)
cv_inner = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=2, test_size=step)

In [65]:
space = dict()
space['constraints'] = [(), constraint_func(X_train)]
space['x0'] = [[0.01]*8,[1]*8]

In [66]:
# models, c, model_mse = mr.Nested_CV(X = X, y = y, model = cls, 
#                                              cv_inner = cv_inner, cv_outer = cv_outer, 
#                                              search_method = 'Grid', space = space)

### Benchmark model: sample mean

In [67]:
# sm_pred, sm_mse = bench.sample_mean(y, "1988-01-01", cv_outer = cv_outer)

### $R^2$ plot

In [68]:
# bench.plot_R2(y_test[::3], c, sm_pred, adjust = False, alpha = 0.8)